<a href="https://www.kaggle.com/code/thanhngan123/segnet?scriptVersionId=233855724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

ver 8 (6), v10

# **Import lib**

In [1]:
import numpy as np
import os, PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

import cv2, json
from tqdm import tqdm 
import matplotlib.pyplot as plt

import random
from scipy.spatial import ConvexHull

# thư viện cần để chạy code nhduong
from skimage.transform import resize
from skimage.io import imread
import time
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation, MaxPooling2D,
    UpSampling2D, Reshape, Permute
)
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split 

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# **Chuẩn bị tập dữ liệu**

In [2]:
# Đọc file Excel
file_path = '/kaggle/input/btxrd-data/classification.xlsx'
df = pd.read_excel(file_path)

# Hiển thị 10 dòng đầu tiên
df.head(10)

,image_id,center,age,gender,neoplasm,tumor_type,img_type,skeletal_type,bones_type,points,area,image_area,tumor_ratio,is_type,image_filename
0,IMG000001,1,48,F,malignant,other mt,frontal,pelvis,hip bone,"[[[2027.9285714285713, 1219.9285714285713], [2...",294967,7701561.0,3.83,test,IMG000001.jpeg
1,IMG000002,1,12,M,malignant,osteosarcoma,frontal,lower limb,tibia,"[[[895.6756756756756, 614.1981981981984], [895...",131041,4972480.0,2.64,train,IMG000002.jpeg
2,IMG000003,1,12,M,malignant,osteosarcoma,lateral,lower limb,tibia,"[[[811.7117117117116, 729.5135135135137], [831...",120650,6900832.0,1.75,train,IMG000003.jpeg
3,IMG000004,1,52,M,malignant,osteosarcoma,frontal,lower limb,foot,"[[[1302.6698113207547, 1734.9056603773583], [1...",91065,7214337.0,1.26,train,IMG000004.jpeg
4,IMG000005,1,52,M,malignant,osteosarcoma,oblique,lower limb,foot,"[[[1294.2446043165467, 1594.1870503597122], [1...",88469,7017152.0,1.26,train,IMG000005.jpeg
5,IMG000006,1,62,M,malignant,osteosarcoma,frontal,lower limb,femur,"[[[591.9130434782609, 1916.3043478260868], [47...",806550,5627824.0,14.33,train,IMG000006.jpeg
6,IMG000007,1,62,M,malignant,osteosarcoma,lateral,lower limb,femur,"[[[868.3636363636363, 420.45454545454544], [92...",592080,5172384.0,11.45,train,IMG000007.jpeg
7,IMG000008,1,6,M,malignant,osteosarcoma,frontal,upper limb,humerus,"[[[585.0, 904.7058823529412], [577.35294117647...",61063,2650000.0,2.30,train,IMG000008.jpeg
8,IMG000009,1,6,M,malignant,osteosarcoma,frontal,upper limb,humerus,"[[[671.6216216216216, 1090.9819819819818], [64...",49362,3085000.0,1.60,train,IMG000009.jpeg
9,IMG000010,1,16,M,malignant,osteosarcoma,frontal,upper limb,humerus,"[[[327.1296296296297, 1343.1604938271603], [36...",130881,4866640.0,2.69,train,IMG000010.jpeg


In [3]:
image_dir = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
annotation_dir = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"

classification_file_path = '/kaggle/input/btxrd-data/classification.xlsx'
train_file_path = '/kaggle/input/btxrd-data/train.xlsx'
val_file_path = '/kaggle/input/btxrd-data/val.xlsx'
test_file_path = '/kaggle/input/btxrd-data/test.xlsx'

processed_image_dir = "/kaggle/working/btxrd_v2/images" 
annotation_dir = "/kaggle/working/btxrd_v2/Annotations" 

In [4]:
df_class = pd.read_excel(classification_file_path)

tumor_types = df_class['tumor_type'].dropna().unique()

summary = {}

for tumor in tumor_types:
    mask = df_class['tumor_type'] == tumor
    counts = df_class[mask].groupby('is_type').size()

    summary[tumor] = {
        'train': int(counts.get('train', 0)),
        'val': int(counts.get('val', 0)),
        'test': int(counts.get('test', 0)),
    }

df_summary = pd.DataFrame.from_dict(summary, orient='index')
df_summary.index.name = 'Tumor Type'

df_summary['total'] = df_summary.sum(axis=1)

print(df_summary)

                          train  val  test  total
Tumor Type                                       
other mt                     29   11     5     45
osteosarcoma                221   53    23    297
synovial osteochondroma      35   13     2     50
giant cell tumor             65   20     8     93
simple bone cyst            153   37    16    206
other bt                     86   22     7    115
osteochondroma              547  138    69    754
multiple osteochondromas    178   66    19    263
osteofibroma                 23   14     7     44


# **Tiền xử lý dữ liệu**
Scale factor: 0.1, Target size: 512x512

In [5]:
SCALE_FACTOR = 0.1 # Scale về 10%
TARGET_SIZE = 512

In [6]:
output_dir = "/kaggle/working/btxrd_v2"
output_image_dir = os.path.join(output_dir, "images")
output_anno_dir = os.path.join(output_dir, "Annotations")

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_anno_dir, exist_ok=True)

MAX_VISUALIZATIONS = 5 # Số lượng ảnh tối đa để trực quan hóa
visualized_count = 0

def get_bounding_box(points):
    if not points:
        return None
    points_array = np.array(points)
    xmin = int(np.min(points_array[:, 0]))
    ymin = int(np.min(points_array[:, 1]))
    xmax = int(np.max(points_array[:, 0]))
    ymax = int(np.max(points_array[:, 1]))
    # Đảm bảo tọa độ không âm
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    return (xmin, ymin, xmax, ymax)

try:
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    if total_images == 0:
        print(f"Không tìm thấy file ảnh nào trong: {image_dir}")
        exit()
    print(f"Tìm thấy {total_images} ảnh để xử lý.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy thư mục ảnh: {image_dir}")
    exit()

Tìm thấy 3746 ảnh để xử lý.


In [7]:
for file in image_files:
    img_path = os.path.join(image_dir, file)
    anno_filename = file.rsplit('.', 1)[0] + '.json'
    anno_path = os.path.join(annotation_dir, anno_filename)

    img_orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img_orig is None:
        # print(f"Không thể đọc ảnh: {file}") # Có thể bỏ comment nếu muốn thấy lỗi cụ thể trong log tqdm
        continue
    orig_height, orig_width = img_orig.shape[:2]

    # Đọc annotation gốc (nếu có)
    annotation_orig = None
    has_annotation = os.path.exists(anno_path)
    if has_annotation:
        try:
            with open(anno_path, "r", encoding="utf-8") as f:
                annotation_orig = json.load(f)
        except Exception as e:
            # print(f"Lỗi khi đọc annotation {anno_filename}: {e}") # Bỏ comment nếu cần debug
            has_annotation = False

    # Chuẩn bị cho trực quan hóa (NẾU cần)
    img_to_draw_orig = None
    img_to_draw_padded = None
    original_bboxes = []
    transformed_bboxes = []

    should_visualize = has_annotation and (visualized_count < MAX_VISUALIZATIONS)

    if should_visualize:
        img_to_draw_orig = cv2.cvtColor(img_orig, cv2.COLOR_GRAY2BGR)
        if annotation_orig and "shapes" in annotation_orig:
             for shape in annotation_orig["shapes"]:
                if "points" in shape and shape["points"]:
                    bbox = get_bounding_box(shape["points"])
                    if bbox:
                        original_bboxes.append(bbox)
                        cv2.rectangle(img_to_draw_orig, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2) # Red

    # Resize ảnh 
    new_width = max(1, int(orig_width * SCALE_FACTOR))
    new_height = max(1, int(orig_height * SCALE_FACTOR))
    img_resized = cv2.resize(img_orig, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # thêm padding
    pad_h = TARGET_SIZE - new_height
    pad_w = TARGET_SIZE - new_width
    top = max(0, pad_h // 2)
    bottom = max(0, pad_h - top)
    left = max(0, pad_w // 2)
    right = max(0, pad_w - left)
    padded_img = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0) 

    # Lưu ảnh đã xử lý
    output_img_path = os.path.join(output_image_dir, file)
    try:
        cv2.imwrite(output_img_path, padded_img)
    except Exception as e:
        # print(f"Lỗi khi lưu ảnh {output_img_path}: {e}") # Bỏ comment nếu cần debug
        continue

    # Xử lý và lưu annotation (nếu có)
    if has_annotation and annotation_orig:
        annotation_new = json.loads(json.dumps(annotation_orig)) # Deep copy
        if "shapes" in annotation_new:
            for shape in annotation_new["shapes"]:
                if "points" in shape and shape["points"]:
                    original_points = shape["points"]
                    new_points_transformed = []
                    for x, y in original_points:
                        new_x = (x * SCALE_FACTOR) + left
                        new_y = (y * SCALE_FACTOR) + top
                        new_points_transformed.append([new_x, new_y])
                    shape["points"] = new_points_transformed

                    if should_visualize:
                        new_bbox = get_bounding_box(new_points_transformed)
                        if new_bbox:
                            transformed_bboxes.append(new_bbox)

            annotation_new["imageWidth"] = TARGET_SIZE
            annotation_new["imageHeight"] = TARGET_SIZE

            output_annotation_path = os.path.join(output_anno_dir, anno_filename)
            try:
                with open(output_annotation_path, "w", encoding="utf-8") as f:
                    json.dump(annotation_new, f, indent=4)
            except Exception as e:
                pass 

            # Trực quan hóa (NẾU cần và đã chuẩn bị)
            if should_visualize and img_to_draw_orig is not None:
                img_to_draw_padded = cv2.cvtColor(padded_img, cv2.COLOR_GRAY2BGR)
                for bbox in transformed_bboxes:
                     cv2.rectangle(img_to_draw_padded, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2) # Green

                fig, axes = plt.subplots(1, 2, figsize=(12, 6))
                axes[0].imshow(img_to_draw_orig)
                axes[0].set_title(f'Original: {file}\nSize: {orig_width}x{orig_height}')
                axes[0].axis('off')
                axes[1].imshow(img_to_draw_padded)
                axes[1].set_title(f'Processed\nSize: {TARGET_SIZE}x{TARGET_SIZE}')
                axes[1].axis('off')
                plt.suptitle(f"Visualization {visualized_count + 1}/{MAX_VISUALIZATIONS}")
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.show()
                visualized_count += 1

print(f"Ảnh và annotation đã xử lý được lưu tại: {output_dir}")

Ảnh và annotation đã xử lý được lưu tại: /kaggle/working/btxrd_v2


In [8]:
# Kiểm tra xem khi xoay ảnh mark có xoay không
processed_image_dir = "/kaggle/working/btxrd_v2/images" 
annotation_dir = "/kaggle/working/btxrd_v2/Annotations" 

rotation_angle = 90  
num_visualizations = 5 
point_color_orig = 'red'  
point_color_rotated = 'lime' 
point_size = 15 

print(f"Kiểm tra xoay ảnh và điểm annotation từ JSON:")
print(f"Thư mục ảnh: {processed_image_dir}")
print(f"Thư mục annotation: {annotation_dir}")
print(f"Góc xoay kiểm tra: {rotation_angle} độ")

# lấy danh sách file ảnh
try:
    all_image_files = [f for f in os.listdir(processed_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not all_image_files:
        print(f"Lỗi: Không tìm thấy file ảnh nào trong {processed_image_dir}")
        exit()
    print(f"\nTìm thấy tổng cộng {len(all_image_files)} ảnh.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy thư mục ảnh: {processed_image_dir}")
    exit()
except Exception as e:
    print(f"Lỗi khi truy cập thư mục ảnh: {e}")
    exit()

# Chọn ngẫu nhiên ảnh để kiểm tra 
# Lọc ra những ảnh có file annotation tương ứng tồn tại
files_with_annotations = []
for img_file in all_image_files:
    json_filename = os.path.splitext(img_file)[0] + '.json'
    json_path = os.path.join(annotation_dir, json_filename)
    if os.path.exists(json_path):
        files_with_annotations.append(img_file)

if not files_with_annotations:
    print("Lỗi: Không tìm thấy cặp ảnh/annotation nào để kiểm tra.")
    exit()

num_to_select = min(num_visualizations, len(files_with_annotations))
if num_to_select < num_visualizations:
     print(f"Cảnh báo: Chỉ tìm thấy {num_to_select} ảnh có annotation, sẽ hiển thị những ảnh này.")

selected_files = random.sample(files_with_annotations, num_to_select)
print(f"Đã chọn ngẫu nhiên {num_to_select} ảnh có annotation để hiển thị kiểm tra xoay.")

# kiểm tra và hiển thị
for i, filename in enumerate(selected_files):
    image_path = os.path.join(processed_image_dir, filename)
    json_filename = os.path.splitext(filename)[0] + '.json'
    json_path = os.path.join(annotation_dir, json_filename)

    print(f"\n--- Ảnh {i+1}/{num_to_select}: {filename} ---")
    print(f"  Annotation: {json_path}")

    img = cv2.imread(image_path, cv2.IMREAD_COLOR) 
    if img is None:
        print(f"  Lỗi: Không thể đọc ảnh {image_path}")
        continue
    h_img, w_img = img.shape[:2]

    annotation = None
    original_points_flat = [] # Danh sách [[x1,y1], [x2,y2], ...]
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            annotation = json.load(f)
        # Trích xuất tất cả các điểm từ tất cả các shape
        if annotation and "shapes" in annotation:
            for shape in annotation["shapes"]:
                if "points" in shape and shape["points"]:
                    original_points_flat.extend(shape["points"])
        if not original_points_flat:
            print("  Cảnh báo: Không tìm thấy 'points' hợp lệ trong file JSON. Bỏ qua hiển thị điểm.")

    except FileNotFoundError:
         print(f"  Lỗi: Không tìm thấy file annotation {json_path}")
         continue 
    except Exception as e:
        print(f"  Lỗi khi đọc hoặc phân tích JSON {json_path}: {e}")
        continue # Bỏ qua nếu lỗi đọc json

    # xoay
    center = (w_img // 2, h_img // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)

    img_rotated = cv2.warpAffine(img, rotation_matrix, (w_img, h_img),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=(0, 0, 0))

    transformed_points_flat = []
    if original_points_flat:
        for x, y in original_points_flat:
            new_x = rotation_matrix[0, 0] * x + rotation_matrix[0, 1] * y + rotation_matrix[0, 2]
            new_y = rotation_matrix[1, 0] * x + rotation_matrix[1, 1] * y + rotation_matrix[1, 2]
            transformed_points_flat.append([new_x, new_y])

    fig, axes = plt.subplots(1, 2, figsize=(15, 7)) # 1 hàng, 2 cột
    
    # vẽ cho ảnh gốc
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"Original Image\n{filename}")
    axes[0].axis('off')
    if annotation and "shapes" in annotation:
        plotted_orig = False
        for shape in annotation["shapes"]:
            if "points" in shape and len(shape["points"]) >= 3: # Cần ít nhất 3 điểm để tạo Convex Hull
                points_orig_np = np.array(shape["points"])
                try:
                    hull = ConvexHull(points_orig_np)
                    # Vẽ các cạnh của Convex Hull
                    for simplex in hull.simplices:
                        axes[0].plot(points_orig_np[simplex, 0], points_orig_np[simplex, 1],
                                     color=point_color_orig, linewidth=2,
                                     label='Original Outline' if not plotted_orig else "")
                        plotted_orig = True # Đảm bảo label chỉ được thêm một lần cho ảnh đầu tiên
                except Exception as e:
                    print(f"  Lỗi khi tính Convex Hull cho ảnh gốc shape: {e}")
                    # Dự phòng: Vẽ các điểm nếu không tính được hull
                    axes[0].scatter(points_orig_np[:, 0], points_orig_np[:, 1], color=point_color_orig, s=5, label='Original Points (Hull Error)' if not plotted_orig else "")
                    plotted_orig = True
    
        if plotted_orig:
            axes[0].legend(loc='upper right')
    
    # vẽ cho ảnh đã xoay
    axes[1].imshow(cv2.cvtColor(img_rotated, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"Rotated Image & Outline ({rotation_angle}°)")
    axes[1].axis('off')
    if annotation and "shapes" in annotation:
        plotted_rotated = False
        for shape_index, shape in enumerate(annotation["shapes"]):
             if "points" in shape and len(shape["points"]) >= 3:
                original_points_shape = shape["points"]
                transformed_points_shape = []
                for x, y in original_points_shape:
                    new_x = rotation_matrix[0, 0] * x + rotation_matrix[0, 1] * y + rotation_matrix[0, 2]
                    new_y = rotation_matrix[1, 0] * x + rotation_matrix[1, 1] * y + rotation_matrix[1, 2]
                    transformed_points_shape.append([new_x, new_y])
    
                if transformed_points_shape:
                    points_transformed_np = np.array(transformed_points_shape)
                    try:
                        if len(points_transformed_np) >= 3:
                             hull_rotated = ConvexHull(points_transformed_np)
                             for simplex in hull_rotated.simplices:
                                 axes[1].plot(points_transformed_np[simplex, 0], points_transformed_np[simplex, 1],
                                              color=point_color_rotated, linewidth=2,
                                              label='Rotated Outline' if not plotted_rotated else "")
                                 plotted_rotated = True
                        elif len(points_transformed_np) > 0 : # Nếu ít hơn 3 điểm, vẽ điểm rời rạc
                            axes[1].scatter(points_transformed_np[:, 0], points_transformed_np[:, 1], color=point_color_rotated, s=5, label='Rotated Points (<3)' if not plotted_rotated else "")
                            plotted_rotated = True
    
                    except Exception as e:
                        print(f"  Lỗi khi tính Convex Hull cho ảnh xoay shape: {e}")
                        # Dự phòng: Vẽ các điểm nếu không tính được hull
                        if len(points_transformed_np) > 0:
                             axes[1].scatter(points_transformed_np[:, 0], points_transformed_np[:, 1], color=point_color_rotated, s=5, label='Rotated Points (Hull Error)' if not plotted_rotated else "")
                             plotted_rotated = True
    
        if plotted_rotated:
            axes[1].legend(loc='upper right')
    
    
    plt.tight_layout()
    plt.show()

Kiểm tra xoay ảnh và điểm annotation từ JSON:
Thư mục ảnh: /kaggle/working/btxrd_v2/images
Thư mục annotation: /kaggle/working/btxrd_v2/Annotations
Góc xoay kiểm tra: 90 độ

Tìm thấy tổng cộng 3746 ảnh.
Lỗi: Không tìm thấy cặp ảnh/annotation nào để kiểm tra.
Cảnh báo: Chỉ tìm thấy 0 ảnh có annotation, sẽ hiển thị những ảnh này.
Đã chọn ngẫu nhiên 0 ảnh có annotation để hiển thị kiểm tra xoay.


# **Load data**

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os
import json
import cv2
import time

# --- Configuration ---
N_CLASSES = 2 # Background, Cancerous
BATCH_SIZE = 8
IMG_SIZE = (512, 512) # Target image size (height, width)
SEED = 42 # For reproducibility

# --- Paths ---
BASE_DATA_DIR = '/kaggle/input/btxrd-data/' # Adapt if needed
EXCEL_PATH = os.path.join(BASE_DATA_DIR, 'classification.xlsx')
PROCESSED_IMAGE_DIR = '/kaggle/working/btxrd_v2/images' # Adapt if needed
ANNOTATION_DIR = '/kaggle/working/btxrd_v2/Annotations' # Adapt if needed
MODEL_SAVE_PATH = 'unet_segmentation_best.keras'

# Create directories if they don't exist (optional, good practice)
os.makedirs(PROCESSED_IMAGE_DIR, exist_ok=True)
os.makedirs(ANNOTATION_DIR, exist_ok=True)

# --- Load Master Metadata ---
print("Loading metadata from:", EXCEL_PATH)
try:
    classification_df = pd.read_excel(EXCEL_PATH)
    print(f"Loaded classification_df: {len(classification_df)} entries")
    # Ensure 'image_id' is string and doesn't contain extensions
    classification_df['image_id'] = classification_df['image_id'].astype(str).str.replace(r'\.(jpg|jpeg|png)$', '', regex=True)
    print("Unique is_type values:", classification_df['is_type'].unique())
except FileNotFoundError:
    print(f"Error: Excel file not found at {EXCEL_PATH}")
    exit() # Or handle appropriately
except Exception as e:
    print(f"Error reading Excel file: {e}")
    exit()

# --- Calculate Mean/Std for Standardization (Requirement 3) ---
# This requires iterating through the training images once beforehand.
# For simplicity in this example, we'll stick to [0, 1] normalization,
# but here's where you'd calculate it if needed.
# train_image_paths_for_stats = [os.path.join(PROCESSED_IMAGE_DIR, f"{img_id}.png") # or .jpg/.jpeg
#                                for img_id in classification_df[classification_df['is_type'] == 'train']['image_id']]
# # Add logic here to load images, calculate mean/std pixel values across the training set
# GLOBAL_MEAN = ... # Calculated mean pixel value(s)
# GLOBAL_STD = ...  # Calculated std dev pixel value(s)
# print(f"Calculated Mean: {GLOBAL_MEAN}, Std: {GLOBAL_STD}") # Placeholder

# --- Augmentation Function (Requirement 1) ---
def augment(image, mask):
    # Geometric Augmentations (apply to both image and mask)
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)

    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)

    # Rotation
    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    # Ensure mask rotation doesn't mess up one-hot encoding if applied later
    # Apply rotation *before* one-hot encoding the mask in parse_fn

    # Photometric Augmentations (apply only to image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # Ensure image values remain in a valid range after augmentation
    image = tf.clip_by_value(image, 0.0, 1.0) # Important if using [0,1] normalization

    return image, mask

# --- Data Loader Function (Requirements 2, 5, 6) ---
def data_loader(image_ids, is_training, batch_size, img_size, n_classes):
    """
    Creates a tf.data.Dataset pipeline for loading images and masks.

    Args:
        image_ids (list or np.array): List of image base names (without extension).
        is_training (bool): Whether this is the training set (enables shuffle, repeat, augment).
        batch_size (int): Batch size.
        img_size (tuple): Target image size (height, width).
        n_classes (int): Number of segmentation classes.

    Returns:
        tf.data.Dataset: Configured dataset pipeline.
    """
    dataset = tf.data.Dataset.from_tensor_slices(image_ids)

    if is_training:
        dataset = dataset.shuffle(len(image_ids), seed=SEED) # Shuffle indices

    def parse_fn(image_id_tensor):
        image_id = tf.strings.as_string(image_id_tensor) # Get string from tensor

        # Construct potential paths (handle multiple extensions if needed)
        # Prioritize png, then jpeg, then jpg - adjust based on your actual data
        img_path_png = tf.strings.join([PROCESSED_IMAGE_DIR, '/', image_id, '.png'])
        img_path_jpeg = tf.strings.join([PROCESSED_IMAGE_DIR, '/', image_id, '.jpeg'])
        img_path_jpg = tf.strings.join([PROCESSED_IMAGE_DIR, '/', image_id, '.jpg'])

        # Determine which path exists (tf.io.is_file or similar might be complex here)
        # A common approach is to try loading, but tf.py_function is cleaner for this check
        def find_image_path(img_id_str):
            img_id_str = img_id_str.decode('utf-8')
            for ext in ['.png', '.jpeg', '.jpg']:
                path = os.path.join(PROCESSED_IMAGE_DIR, img_id_str + ext)
                if os.path.exists(path):
                    return path.encode('utf-8')
            # Return a dummy path or raise error if no image found
            print(f"Warning: Image file not found for ID: {img_id_str}")
            # Returning one of the attempts; TF load will fail later if it doesn't exist
            return os.path.join(PROCESSED_IMAGE_DIR, img_id_str + ".png").encode('utf-8') # Or handle error

        image_path = tf.py_function(find_image_path, [image_id], tf.string)
        image_path.set_shape([]) # Set shape for py_function output

        # Load Image
        image = tf.io.read_file(image_path)
        try:
            # Try decoding as PNG first, then JPEG if needed
            image = tf.image.decode_image(image, channels=1, expand_animations=False) # Handles png, jpg, bmp, gif
        except tf.errors.InvalidArgumentError: # Fallback if decode_image fails (e.g., corrupted)
             print(f"Warning: Could not decode image {image_path}. Using zeros.")
             image = tf.zeros([img_size[0], img_size[1], 1], dtype=tf.uint8)


        image = tf.image.resize(image, img_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1] (Requirement 2)
        # If using standardization (Req 3):
        # image = (tf.cast(image, tf.float32) - GLOBAL_MEAN) / GLOBAL_STD

        # Load Mask using tf.numpy_function for complex JSON parsing
        annotation_path = tf.strings.join([ANNOTATION_DIR, '/', image_id, '.json'])

        def get_mask(annotation_path_bytes):
            path = annotation_path_bytes.decode('utf-8')
            # Initialize mask as background
            mask = np.zeros(img_size, dtype=np.uint8) # Use target img_size
            if not os.path.exists(path):
                # print(f"Annotation file not found: {path}. Returning zero mask.") # Can be noisy
                pass # Keep zero mask
            else:
                try:
                    with open(path) as f:
                        data = json.load(f)
                    
                    # Get original image dimensions for scaling points
                    orig_h = data.get('imageHeight', img_size[0])
                    orig_w = data.get('imageWidth', img_size[1])
                    if orig_h == 0 or orig_w == 0: # Handle potential zero dimensions
                         orig_h, orig_w = img_size[0], img_size[1]


                    scale_y = img_size[0] / orig_h
                    scale_x = img_size[1] / orig_w

                    for shape in data.get('shapes', []):
                        if shape.get('label', '').lower() == 'cancerous' or True: # Adjust label if needed, or take all polygons
                           if 'points' not in shape or len(shape['points']) < 3:
                               continue
                           pts = np.array(shape['points'], dtype=np.float32)
                           # Scale points to the target IMG_SIZE
                           pts[:, 0] *= scale_x
                           pts[:, 1] *= scale_y
                           pts = np.round(pts).astype(np.int32)
                           # Ensure points are within bounds after scaling/rounding
                           pts[:, 0] = np.clip(pts[:, 0], 0, img_size[1] - 1)
                           pts[:, 1] = np.clip(pts[:, 1], 0, img_size[0] - 1)
                           # Fill polygon with class index 1 (assuming 0 is background)
                           cv2.fillPoly(mask, [pts], 1)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON: {path}. Returning zero mask.")
                except Exception as e:
                    print(f"Error processing annotation {path}: {e}. Returning zero mask.")

            # Add channel dimension
            mask = mask[..., np.newaxis]
            return mask.astype(np.int32) # Return integer mask before one-hot

        # Use tf.numpy_function to run the Python code
        mask_int = tf.numpy_function(get_mask, [annotation_path], tf.int32)
        mask_int.set_shape((*img_size, 1)) # Explicitly set shape

        # --- Apply Augmentations (if training) ---
        if is_training:
            # Apply geometric augmentations before one-hot encoding mask
            image, mask_int = augment(image, mask_int) # Pass integer mask to augment

        # --- One-Hot Encode Mask (Requirement 5) ---
        # Squeeze the channel dim before one-hot
        mask = tf.one_hot(tf.squeeze(mask_int, axis=-1), depth=n_classes, dtype=tf.float32)
        mask.set_shape((*img_size, n_classes)) # Ensure shape consistency

        # Final check on image shape (sometimes resize might lose channel dim for grayscale)
        image.set_shape((*img_size, 1))

        return image, mask

    # Configure dataset pipeline
    dataset = dataset.map(parse_fn, num_parallel_calls=tf.data.AUTOTUNE)

    if is_training:
        # Repeat *before* batching for effective shuffling across epochs
        dataset = dataset.repeat()

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE) # Prefetch for performance

    return dataset

# --- Split Data based on 'is_type' column (Requirement 6) ---
train_ids = classification_df[classification_df['is_type'] == 'train']['image_id'].tolist()
val_ids = classification_df[classification_df['is_type'] == 'val']['image_id'].tolist()
test_ids = classification_df[classification_df['is_type'] == 'test']['image_id'].tolist()

print(f"Train samples: {len(train_ids)}")
print(f"Validation samples: {len(val_ids)}")
print(f"Test samples: {len(test_ids)}")

# --- Create Datasets using the data_loader ---
train_ds = data_loader(train_ids, is_training=True, batch_size=BATCH_SIZE, img_size=IMG_SIZE, n_classes=N_CLASSES)
val_ds = data_loader(val_ids, is_training=False, batch_size=BATCH_SIZE, img_size=IMG_SIZE, n_classes=N_CLASSES)
test_ds = data_loader(test_ids, is_training=False, batch_size=BATCH_SIZE, img_size=IMG_SIZE, n_classes=N_CLASSES)

# --- Verify dataset output shapes (optional but recommended) ---
print("Checking dataset element spec...")
for image_batch, mask_batch in train_ds.take(1):
    print("Train Image batch shape:", image_batch.shape) # (BATCH_SIZE, H, W, 1)
    print("Train Mask batch shape:", mask_batch.shape)   # (BATCH_SIZE, H, W, N_CLASSES)
for image_batch, mask_batch in val_ds.take(1):
    print("Validation Image batch shape:", image_batch.shape)
    print("Validation Mask batch shape:", mask_batch.shape)


# --- Loss Functions and Metrics ---
def dice_coef(y_true, y_pred, smooth=1e-6):
    # y_true shape: (batch, h, w, n_classes), y_pred shape: (batch, h, w, n_classes)
    # Flatten spatial dimensions
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])

    # Calculate intersection and union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0) # Shape: (n_classes,)
    union = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0) # Shape: (n_classes,)

    dice = (2. * intersection + smooth) / (union + smooth) # Shape: (n_classes,)

    # Typically interested in the foreground class(es) Dice
    # Option 1: Average Dice across all classes (including background)
    # return tf.reduce_mean(dice)
    # Option 2: Dice for the foreground class (index 1) - Common for binary
    return dice[1] # Return Dice coefficient specifically for class 1 (cancerous)

def dice_loss(y_true, y_pred, smooth=1e-6):
     # Uses the same logic as dice_coef but returns 1 - dice
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    # Option 1: Average loss over classes
    # return 1.0 - tf.reduce_mean(dice_per_class)
    # Option 2: Loss based on foreground class Dice
    return 1.0 - dice_per_class[1] # Loss based on class 1


def iou_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0) - intersection
    iou_per_class = (intersection + smooth) / (union + smooth)
    # Option 1: Average IoU
    # return tf.reduce_mean(iou_per_class)
    # Option 2: Foreground IoU
    return iou_per_class[1] # IoU for class 1


# Focal loss implementation remains the same if desired
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32) # Ensure y_true is float
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1. - tf.keras.backend.epsilon())

        # Calculate cross-entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate focal loss weights
        loss_weight = alpha * y_true * tf.pow((1 - y_pred), gamma)

        # Apply weights to cross-entropy
        focal_loss = loss_weight * cross_entropy

        # Sum across classes and reduce mean across batch/pixels
        # return tf.reduce_mean(tf.reduce_sum(focal_loss, axis=-1)) # Mean over pixels/batch
        # Sum over classes, then take the mean over batch/pixels
        return tf.reduce_mean(tf.reduce_sum(focal_loss, axis=-1))

    return focal_loss_fixed

# Combined Loss (Example: Dice + Focal)
def combined_loss(y_true, y_pred):
     # Weighting can be adjusted (e.g., 0.5 * dice + 0.5 * focal)
    return dice_loss(y_true, y_pred) + focal_loss(gamma=2.0, alpha=0.25)(y_true, y_pred)
    # return dice_loss(y_true, y_pred) # Or just use Dice Loss


# --- Build U-Net Model ---
# The U-Net definition from the original code is kept, as it's a standard structure.
def unet_model(input_shape, num_classes):
    inputs = layers.Input(input_shape)

    # Contracting Path (Encoder)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1) # Explicit MaxPooling

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.BatchNormalization()(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)
    c5 = layers.BatchNormalization()(c5)

    # Expansive Path (Decoder)
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    # u6 = layers.Resizing(c4.shape[1], c4.shape[2])(u6) # Optional if shapes mismatch slightly
    u6 = layers.Concatenate()([u6, c4])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)
    c6 = layers.BatchNormalization()(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    # u7 = layers.Resizing(c3.shape[1], c3.shape[2])(u7)
    u7 = layers.Concatenate()([u7, c3])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)
    c7 = layers.BatchNormalization()(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    # u8 = layers.Resizing(c2.shape[1], c2.shape[2])(u8)
    u8 = layers.Concatenate()([u8, c2])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)
    c8 = layers.BatchNormalization()(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    # u9 = layers.Resizing(c1.shape[1], c1.shape[2])(u9)
    u9 = layers.Concatenate()([u9, c1])
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c9)
    c9 = layers.BatchNormalization()(c9)

    # Output layer: Use softmax for multi-class (even if N=2 with one-hot)
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# --- Instantiate and Compile Model ---
tf.keras.backend.clear_session() # Clear previous models from memory
model = unet_model(input_shape=(*IMG_SIZE, 1), num_classes=N_CLASSES)

# Use combined loss or just dice_loss
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#               loss=combined_loss,
#               metrics=['accuracy', dice_coef, iou_coef])

# Compile with Dice Loss only (often a good starting point)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=dice_loss, # Use the foreground dice loss
              metrics=['accuracy', dice_coef, iou_coef]) # Monitor foreground Dice/IoU


model.summary()

# --- Callbacks ---
checkpoint_cb = ModelCheckpoint(
    MODEL_SAVE_PATH,
    save_best_only=True,
    monitor='val_dice_coef', # Monitor validation Dice coefficient for the foreground class
    mode='max',             # Save the model with the highest validation Dice
    verbose=1
)
lr_cb = ReduceLROnPlateau(
    monitor='val_loss',     # Reduce LR if validation loss plateaus
    factor=0.2,             # Reduce LR by factor of 5
    patience=5,             # Number of epochs with no improvement before reducing LR
    min_lr=1e-6,            # Minimum learning rate
    verbose=1
)
early_cb = EarlyStopping(
    monitor='val_dice_coef', # Stop if validation Dice stops improving
    patience=10,            # Number of epochs to wait for improvement
    mode='max',             # Looking for maximum Dice
    restore_best_weights=True, # Restore weights from the best epoch
    verbose=1
)

# --- Training (Requirement 7 - Using model.fit, fulfilling other parts) ---
# Calculate steps per epoch and validation steps
steps_per_epoch = len(train_ids) // BATCH_SIZE
if steps_per_epoch == 0:
    steps_per_epoch = 1 # Ensure at least one step even if dataset is small
validation_steps = len(val_ids) // BATCH_SIZE
if validation_steps == 0:
    validation_steps = 1

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

N_EPOCHS = 50 # Set desired number of epochs

print("Starting training...")
start_time = time.time()

# Fit the model using the prepared tf.data datasets
history = model.fit(
    train_ds,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, lr_cb, early_cb],
    verbose=1 # Set to 1 or 2 for progress updates
)

end_time = time.time()
print(f"Training finished.")
print(f"Total Training time: {(end_time - start_time)/60:.2f} minutes")

# --- Evaluation on Test Set (Optional) ---
print("Evaluating on test set...")
# Load the best weights saved by ModelCheckpoint
model.load_weights(MODEL_SAVE_PATH)

test_steps = len(test_ids) // BATCH_SIZE
if test_steps == 0:
    test_steps = 1

test_loss, test_accuracy, test_dice, test_iou = model.evaluate(test_ds, steps=test_steps, verbose=1)
print(f"\nTest Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Dice Coefficient (Foreground): {test_dice:.4f}")
print(f"Test IoU (Foreground): {test_iou:.4f}")

# --- Plot Training History (Optional) ---
import matplotlib.pyplot as plt

def plot_history(history):
    # Plot Dice Coefficient and Validation Dice Coefficient
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['dice_coef'], label='Training Dice Coef')
    plt.plot(history.history['val_dice_coef'], label='Validation Dice Coef')
    plt.title('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.grid(True)

    # Plot Loss and Validation Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_history(history)

Loading metadata from: /kaggle/input/btxrd-data/classification.xlsx
Loaded classification_df: 3746 entries
Unique is_type values: ['test' 'train' 'val' nan]
Train samples: 2677
Validation samples: 744
Test samples: 298
Checking dataset element spec...


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:2 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::Prefetch::BatchV2::ForeverRepeat[0]::ParallelMapV2: AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'decode'
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*arg

In [ ]:
abc

nháp từ đây tới dưới

In [ ]:
N_CLASSES = 2
BATCH_SIZE = 8
IMG_SIZE = (512, 512)

# Đọc dữ liệu từ Excel
classification_df = pd.read_excel('/kaggle/input/btxrd-data/classification.xlsx')
val_df = pd.read_excel('/kaggle/input/btxrd-data/val.xlsx')
test_df = pd.read_excel('/kaggle/input/btxrd-data/test.xlsx')

processed_image_dir = '/kaggle/working/btxrd_v2/images'
annotation_dir = '/kaggle/working/btxrd_v2/Annotations'

In [ ]:
# Hàm xác định phần mở rộng của ảnh
def resolve_image_extension(df, image_dir):
    def get_filename(image_id):
        # image_id đã bao gồm đuôi .jpeg hoặc .jpg
        path = os.path.join(image_dir, image_id)
        return image_id if os.path.exists(path) else None

    df['image_filename'] = df['image_id'].apply(get_filename)
    df = df[df['image_filename'].notna()].reset_index(drop=True)
    return df

train_df = resolve_image_extension(train_df, processed_image_dir)
val_df = resolve_image_extension(val_df, processed_image_dir)
test_df = resolve_image_extension(test_df, processed_image_dir)

print("train_df sau khi resolve:", len(train_df))
print("val_df sau khi resolve:", len(val_df))
print("test_df sau khi resolve:", len(test_df))


In [ ]:
def augment(image, mask):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)

    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)

    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    mask = tf.image.rot90(mask, k)

    # Brightness and contrast (only on image)
    if tf.random.uniform(()) > 0.5:
        image = tf.image.random_brightness(image, max_delta=0.2)
    if tf.random.uniform(()) > 0.5:
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)

    return image, mask

In [ ]:
# Data loader
def data_loader(df, is_training):
    print("data_loader: input df length =", len(df))
    def parse_fn(image_filename):
        image_path = tf.strings.join([processed_image_dir, '/', image_filename])
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=1)
        image = tf.image.resize(image, IMG_SIZE)
        image = tf.cast(image, tf.float32) / 255.0

        image_id = tf.strings.regex_replace(image_filename, r'\.(jpg|jpeg)$', '')
        annotation_path = tf.strings.join([annotation_dir, '/', image_id, '.json'])

        # Ground Truth Mask
        def get_mask(annotation_path_str):
            path = annotation_path_str.decode('utf-8')
            if not os.path.exists(path):
                return np.zeros((*IMG_SIZE, 1), dtype=np.float32)
            with open(path) as f:
                data = json.load(f)
            mask = np.zeros(IMG_SIZE, dtype=np.uint8)
            for shape in data.get('shapes', []):
                if 'points' not in shape:
                    continue
                pts = np.array(shape['points'], dtype=np.float32)
                if pts.shape[0] < 3:
                    continue
                pts[:, 0] = pts[:, 0] * IMG_SIZE[1] / data['imageWidth']
                pts[:, 1] = pts[:, 1] * IMG_SIZE[0] / data['imageHeight']
                pts = np.round(pts).astype(np.int32)
                cv2.fillPoly(mask, [pts], 1)
            return mask[..., np.newaxis].astype(np.float32)

        mask = tf.numpy_function(get_mask, [annotation_path], tf.float32)
        mask.set_shape((*IMG_SIZE, 1))
        mask = tf.cast(mask, tf.int32)
        mask = tf.one_hot(tf.squeeze(mask), depth=N_CLASSES)

         # --- Augmentation ---
        image, mask = augment(image, mask)

        return image, mask

    df = df.sample(frac=1).reset_index(drop=True) if is_training else df
    dataset = tf.data.Dataset.from_tensor_slices(df['image_filename'].values)
    dataset = dataset.map(parse_fn, num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(100).repeat().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    else:
        dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
def load_dataset_from_index(index_list, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices(index_list)

    def load_image_mask(idx):
        image_id = classification_df.iloc[int(idx.numpy())]['image_id']
        return image_id.encode()

    def parse_fn(image_id):
        image_id_str = image_id.numpy().decode("utf-8")
        
        image_path = os.path.join(processed_image_dir, f"{image_id_str}.png")
        mask_path = os.path.join(annotation_dir, f"{image_id_str}.json")

        image = load_grayscale_image(image_path)  # bạn đã có
        mask = load_mask_from_json(mask_path)     # bạn đã có

        return image, mask

    def map_fn(idx):
        image_id = tf.py_function(load_image_mask, [idx], tf.string)
        return tf.py_function(parse_fn, [image_id], [tf.float32, tf.float32])

    dataset = dataset.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)

    if is_training:
        dataset = dataset.shuffle(100).repeat()

    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
# Lấy index dòng tương ứng với 'train' từ classification.xlsx
train_indices = classification_df[classification_df['is_type'] == 'train'].index.tolist()
# Tạo train dataset từ index
train_ds = load_dataset_from_index(train_indices, is_training=True)

In [ ]:
train_ds = data_loader(train_df, is_training=True)
val_ds = data_loader(val_df, is_training=False)

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f + y_pred_f, axis=0)
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - tf.reduce_mean(dice)

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f + y_pred_f, axis=0)
    dice = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice)

def iou_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1, N_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, N_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f + y_pred_f - y_true_f * y_pred_f, axis=0)
    iou = (intersection + smooth) / (union + smooth)
    return tf.reduce_mean(iou)

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.reshape(y_true, [-1, N_CLASSES])
        y_pred = tf.reshape(y_pred, [-1, N_CLASSES])
        epsilon = 1e-6
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + focal_loss()(y_true, y_pred)


In [ ]:
# Build U-Net model
def unet_model():
    inputs = layers.Input((*IMG_SIZE, 1))

    x = layers.Conv2D(32, (3, 3), strides=2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    skip1 = x

    x = layers.Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    skip2 = x

    x = layers.Conv2D(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    skip3 = x

    x = layers.Conv2D(256, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Resizing(skip3.shape[1], skip3.shape[2])(x)
    x = layers.Concatenate()([x, skip3])
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Resizing(skip2.shape[1], skip2.shape[2])(x)
    x = layers.Concatenate()([x, skip2])
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Resizing(skip1.shape[1], skip1.shape[2])(x)
    x = layers.Concatenate()([x, skip1])
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Conv2D(N_CLASSES, (1, 1), activation='softmax')(x)

    return models.Model(inputs, outputs)

model = unet_model()
# model.compile(optimizer='adam', loss=dice_loss, metrics=['accuracy', dice_coef, iou_coef])
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy', dice_coef, iou_coef])

model.summary()

In [ ]:
# Callbacks
checkpoint_cb = ModelCheckpoint('unet_v2.keras', save_best_only=True, monitor='val_loss', mode='min')
lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

steps_per_epoch = max(1, len(train_df) // BATCH_SIZE)
validation_steps = max(1, len(val_df) // BATCH_SIZE)

train_ds = data_loader(train_df, is_training=True).repeat()
val_ds = data_loader(val_df, is_training=False)

start_time = time.time()
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    # validation_steps=validation_steps,
    callbacks=[checkpoint_cb, lr_cb, early_cb]
)
end_time = time.time()
print(f"Training time: {(end_time - start_time)/60:.2f} minutes")

nahsp từ đầu tới đây

In [ ]:
# Evaluate Model
score = model.evaluate(x_val, y_val, verbose=0)
print(f'Validation loss: {score[0]:.4f}')
print(f'Validation accuracy: {score[1]:.4f}')

In [ ]:
# Visualize Prediction 
random_index = np.random.randint(0, len(x_val))
input_image = x_val[random_index]
true_mask_one_hot = y_val_orig[random_index]

prediction_reshaped = model.predict(np.expand_dims(input_image, axis=0))
prediction = prediction_reshaped.reshape((MODEL_IMG_H, MODEL_IMG_W, N_CLASSES))
predicted_mask = np.argmax(prediction, axis=-1)
true_mask = np.argmax(true_mask_one_hot, axis=-1)

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(input_image[:, :, 0], cmap='gray')
plt.title('Input Image')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(true_mask, cmap='gray')
plt.title('True Mask (Tumor=1)')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask (Tumor=1)')
plt.axis('off')
plt.tight_layout()
plt.show()